# Test de Normalisation des Images IEFCe notebook teste les méthodes de normalisation sur **5 images par classe** depuis le dataset **train_val**.**Méthodes testées:**- Flat-field- Reinhard- Histogram Matching- Macenko- Pipeline (FF+Reinhard)**Résultats générés:**- Visualisations comparatives par image- Tableau des résultats globaux avec **score combiné**- Métriques détaillées (CSV)- Rapport texte complet

## 1. Installation des dépendances

In [ ]:
!pip install numpy opencv-python pillow scikit-image scipy matplotlib tqdm pandas

## 2. Montage de Google Drive

In [ ]:
from google.colab import drivedrive.mount('/content/drive')

## 3. Configuration des chemins

In [ ]:
import os# Chemin de base dans Google DriveBASE_PATH = "/content/drive/MyDrive/CERPAD/Data_Basse_test_Normalisation/Test_Normalisation"# Chemin vers le module de normalisationNORMALIZATION_MODULE = os.path.join(BASE_PATH, "normalization_ief.py")# Chemin vers le dataset (train_val)DATASET_PATH = os.path.join(BASE_PATH, "Data_paper_TrainVal_Test")TRAIN_VAL_PATH = os.path.join(DATASET_PATH, "train_val")# Répertoire de sortieOUTPUT_DIR = "/content/test_normalization_results"# Nombre d'images par classeSAMPLES_PER_CLASS = 5print(f"✓ Chemin du module: {NORMALIZATION_MODULE}")print(f"✓ Chemin du dataset: {TRAIN_VAL_PATH}")print(f"✓ Répertoire de sortie: {OUTPUT_DIR}")

## 4. Vérification de la structure

In [ ]:
# Vérifier que les fichiers existentif os.path.exists(NORMALIZATION_MODULE):    print(f"✓ Module trouvé: {NORMALIZATION_MODULE}")else:    print(f"❌ Module non trouvé: {NORMALIZATION_MODULE}")if os.path.exists(TRAIN_VAL_PATH):    print(f"✓ Dataset trouvé: {TRAIN_VAL_PATH}")    # Lister les classes    classes = [d for d in os.listdir(TRAIN_VAL_PATH)                if os.path.isdir(os.path.join(TRAIN_VAL_PATH, d))]    print(f"✓ Classes trouvées: {classes}")else:    print(f"❌ Dataset non trouvé: {TRAIN_VAL_PATH}")

## 5. Chargement du module de normalisation

In [ ]:
import syssys.path.insert(0, BASE_PATH)# Importer le modulefrom normalization_ief import IEFNormalizerprint("✓ Module normalization_ief importé avec succès")

## 6. Imports et fonctions utilitaires

In [ ]:
import numpy as npfrom PIL import Imagefrom pathlib import Pathimport matplotlib.pyplot as pltfrom matplotlib.gridspec import GridSpecfrom skimage.metrics import structural_similarity as ssimfrom skimage.metrics import peak_signal_noise_ratio as psnrimport pandas as pdfrom collections import defaultdictdef load_image(image_path: str) -> np.ndarray:    """Charge une image depuis un fichier"""    img = Image.open(image_path)    if img.mode != 'RGB':        img = img.convert('RGB')    return np.array(img)def get_class_images(directory: str, max_samples: int = 5) -> list:    """Récupère un échantillon d'images d'une classe"""    if not os.path.isdir(directory):        return []    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}    image_paths = []    for file in sorted(os.listdir(directory)):        if Path(file).suffix.lower() in image_extensions:            image_paths.append(os.path.join(directory, file))            if len(image_paths) >= max_samples:                break    return image_pathsdef find_all_classes(split_path: str) -> dict:    """Trouve toutes les classes dans le split du dataset"""    classes = {}    if not os.path.isdir(split_path):        return classes    for item in os.listdir(split_path):        item_path = os.path.join(split_path, item)        if os.path.isdir(item_path):            classes[item] = item_path    return classesdef calculate_metrics(original: np.ndarray, normalized: np.ndarray) -> dict:    """Calcule les métriques de qualité"""    try:        ssim_value = ssim(original, normalized, channel_axis=2, data_range=255)    except:        ssim_value = 0.0    try:        psnr_value = psnr(original, normalized, data_range=255)    except:        psnr_value = 0.0    orig_std = np.std(original, axis=(0, 1))    norm_std = np.std(normalized, axis=(0, 1))    variance_reduction = {        'R': (orig_std[0] - norm_std[0]) / orig_std[0] * 100 if orig_std[0] > 0 else 0,        'G': (orig_std[1] - norm_std[1]) / orig_std[1] * 100 if orig_std[1] > 0 else 0,        'B': (orig_std[2] - norm_std[2]) / orig_std[2] * 100 if orig_std[2] > 0 else 0    }    return {        'SSIM': ssim_value,        'PSNR': psnr_value,        'variance_reduction': variance_reduction    }def calculate_image_stats(image: np.ndarray) -> dict:    """Calcule les statistiques d'une image"""    stats = {}    for i, channel in enumerate(['R', 'G', 'B']):        channel_data = image[:, :, i]        stats[f'{channel}_mean'] = np.mean(channel_data)        stats[f'{channel}_std'] = np.std(channel_data)        stats[f'{channel}_var'] = np.var(channel_data)    stats['global_mean'] = np.mean(image)    stats['global_std'] = np.std(image)    stats['global_var'] = np.var(image)    return statsdef calculate_class_coherence(stats_list: list) -> dict:    """Calcule la cohérence statistique d'une classe"""    if not stats_list:        return {}    df = pd.DataFrame(stats_list)    coherence = {}    numeric_cols = df.select_dtypes(include=[np.number]).columns    for col in numeric_cols:        coherence[f'{col}_std_across_images'] = df[col].std()        coherence[f'{col}_mean_across_images'] = df[col].mean()        coherence[f'{col}_cv'] = df[col].std() / df[col].mean() if df[col].mean() != 0 else 0    return coherenceprint("✓ Fonctions utilitaires définies")

## 7. Fonction de visualisation

In [ ]:
def create_comparison_visualization(original: np.ndarray,                                    results: dict,                                   image_name: str,                                   output_path: str):    """Crée une visualisation comparative de toutes les méthodes"""    n_methods = len(results)    n_cols = 3    n_rows = (n_methods + 2) // n_cols + 1        fig = plt.figure(figsize=(18, 6 * n_rows))    gs = GridSpec(n_rows, n_cols, figure=fig, hspace=0.3, wspace=0.3)        # Image originale    ax = fig.add_subplot(gs[0, 0])    ax.imshow(original)    ax.set_title('Image Originale', fontsize=14, fontweight='bold', pad=10)    ax.axis('off')        # Métriques de l'original    orig_mean = np.mean(original, axis=(0, 1))    orig_std = np.std(original, axis=(0, 1))    metrics_text = f"Mean: {orig_mean.mean():.1f}\nStd: {orig_std.mean():.1f}"    ax.text(0.02, 0.98, metrics_text, transform=ax.transAxes,            verticalalignment='top', fontsize=10,            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))        # Espace vide pour l'alignement    ax = fig.add_subplot(gs[0, 1])    ax.axis('off')    ax = fig.add_subplot(gs[0, 2])    ax.axis('off')        # Images normalisées    row = 1    col = 0        for method_name, normalized in results.items():        if row * n_cols + col >= n_rows * n_cols:            break                    ax = fig.add_subplot(gs[row, col])        ax.imshow(normalized)        ax.set_title(f'{method_name}', fontsize=12, fontweight='bold', pad=10)        ax.axis('off')                # Calculer les métriques        metrics = calculate_metrics(original, normalized)                # Afficher les métriques        metrics_text = (            f"SSIM: {metrics['SSIM']:.3f}\n"            f"PSNR: {metrics['PSNR']:.1f} dB\n"            f"Var↓: {metrics['variance_reduction']['R']:.1f}%"        )        ax.text(0.02, 0.98, metrics_text, transform=ax.transAxes,                verticalalignment='top', fontsize=9,                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))                col += 1        if col >= n_cols:            col = 0            row += 1        # Titre général    fig.suptitle(f'Comparaison des Méthodes de Normalisation\n{image_name}',                  fontsize=16, fontweight='bold', y=0.995)        plt.savefig(output_path, dpi=150, bbox_inches='tight')    plt.close()print("✓ Fonction de visualisation définie")

## 8. Fonction de test par classe

In [ ]:
def test_class_samples(class_name: str, class_dir: str,                       output_dir: str,                      max_samples: int = 5,                      flat_field_sigma: float = 50.0,                      return_stats: bool = True):    """Teste les méthodes de normalisation sur un échantillon d'une classe"""    print(f"\n{'='*70}")    print(f"Classe: {class_name}")    print(f"{'='*70}")        # Récupérer les images de la classe    image_paths = get_class_images(class_dir, max_samples=max_samples)        if not image_paths:        print(f"  ⚠ Aucune image trouvée dans {class_dir}")        return {}        print(f"  Images trouvées: {len(image_paths)}")        # Charger les images    images = []    for path in image_paths:        try:            img = load_image(path)            images.append((path, img))        except Exception as e:            print(f"  ⚠ Erreur lors du chargement de {path}: {e}")        if not images:        print(f"  ⚠ Aucune image valide chargée")        return {}        # Sélectionner une image cible (première image de l'échantillon)    sample_images = [img for _, img in images]    if len(sample_images) > 0:        target_image = sample_images[0]    else:        print(f"  ⚠ Aucune image valide pour la référence")        return {}        # Initialiser le normaliseur    normalizer = IEFNormalizer()    normalizer.set_target(target_image)        # Créer le répertoire de sortie pour cette classe    class_output_dir = os.path.join(output_dir, class_name)    os.makedirs(class_output_dir, exist_ok=True)        # Dictionnaire pour stocker les statistiques    class_stats = {}        # Traiter chaque image    for img_path, original in images:        image_name = Path(img_path).stem                print(f"\n  Traitement: {image_name}")                # Appliquer toutes les méthodes        results = {}                # 1. Flat-field        try:            normalized_ff = normalizer.flat_field_correction(original, sigma=flat_field_sigma)            results['Flat-field'] = normalized_ff            print(f"    ✓ Flat-field")        except Exception as e:            print(f"    ✗ Flat-field: {e}")                # 2. Reinhard        try:            normalized_reinhard = normalizer.reinhard_normalization(original, target_image=target_image)            results['Reinhard'] = normalized_reinhard            print(f"    ✓ Reinhard")        except Exception as e:            print(f"    ✗ Reinhard: {e}")                # 3. Histogram Matching        try:            normalized_hist = normalizer.histogram_matching(original, target_image=target_image)            results['Histogram Matching'] = normalized_hist            print(f"    ✓ Histogram Matching")        except Exception as e:            print(f"    ✗ Histogram Matching: {e}")                # 4. Macenko        try:            normalized_macenko = normalizer.macenko_normalization(original, target_image=target_image)            results['Macenko'] = normalized_macenko            print(f"    ✓ Macenko")        except Exception as e:            print(f"    ✗ Macenko: {e}")                # 5. Pipeline combiné        try:            normalized_pipeline = normalizer.normalize_pipeline(                original,                methods=['flat_field', 'reinhard'],                target_image=target_image,                flat_field_sigma=flat_field_sigma            )            results['Pipeline (FF+Reinhard)'] = normalized_pipeline            print(f"    ✓ Pipeline combiné")        except Exception as e:            print(f"    ✗ Pipeline: {e}")                # Créer la visualisation        output_path = os.path.join(class_output_dir, f"{image_name}_comparison.png")        create_comparison_visualization(original, results, image_name, output_path)        print(f"    ✓ Visualisation sauvegardée")                # Sauvegarder aussi les images individuelles        for method_name, normalized in results.items():            method_dir = os.path.join(class_output_dir, method_name.replace(' ', '_'))            os.makedirs(method_dir, exist_ok=True)            output_img_path = os.path.join(method_dir, f"{image_name}.jpg")            Image.fromarray(normalized).save(output_img_path, quality=95)                # Calculer les statistiques pour le tableau global        if return_stats:            orig_stats = calculate_image_stats(original)            for method_name, normalized in results.items():                norm_stats = calculate_image_stats(normalized)                # Mapper les noms d'affichage vers les noms de méthode                method_mapping = {                    'Flat-field': 'flat_field',                    'Reinhard': 'reinhard',                    'Histogram Matching': 'histogram_matching',                    'Macenko': 'macenko',                    'Pipeline (FF+Reinhard)': 'pipeline_ff_reinhard'                }                method_key = method_mapping.get(method_name, method_name.lower().replace(' ', '_'))                if method_key not in class_stats:                    class_stats[method_key] = {'original': [], 'normalized': []}                class_stats[method_key]['original'].append(orig_stats)                class_stats[method_key]['normalized'].append(norm_stats)        # Calculer les métriques globales pour cette classe    if return_stats and class_stats:        metrics_by_method = {}        for method_key, stats_data in class_stats.items():            if stats_data['original'] and stats_data['normalized']:                # Réduction de variance                orig_vars = [s['global_var'] for s in stats_data['original']]                norm_vars = [s['global_var'] for s in stats_data['normalized']]                orig_var_mean = np.mean(orig_vars)                norm_var_mean = np.mean(norm_vars)                variance_reduction = (orig_var_mean - norm_var_mean) / orig_var_mean * 100 if orig_var_mean > 0 else 0                                # Amélioration de cohérence                coherence_orig = calculate_class_coherence(stats_data['original'])                coherence_norm = calculate_class_coherence(stats_data['normalized'])                orig_std_of_means = coherence_orig.get('global_mean_std_across_images', 0)                norm_std_of_means = coherence_norm.get('global_mean_std_across_images', 0)                coherence_improvement = (orig_std_of_means - norm_std_of_means) / orig_std_of_means * 100 if orig_std_of_means > 0 else 0                                metrics_by_method[method_key] = {                    'variance_reduction_%': variance_reduction,                    'coherence_improvement_%': coherence_improvement,                    'mean_std_reduction': orig_std_of_means - norm_std_of_means                }        return metrics_by_method        return {}print("✓ Fonction de test par classe définie")

## 9. Fonction de génération du tableau global

In [ ]:
def generate_global_comparison_table(all_stats: dict, output_dir: str):    """Génère le tableau des résultats globaux avec score combiné"""    print(f"\n{'='*70}")    print(f"GÉNÉRATION DU TABLEAU DES RÉSULTATS GLOBAUX")    print(f"{'='*70}\n")        # Préparer les données pour le DataFrame    comparison_data = []        for class_name, methods_data in all_stats.items():        for method, metrics in methods_data.items():            comparison_data.append({                'Classe': class_name,                'Méthode': method,                'Réduction_Variance_%': metrics.get('variance_reduction_%', 0),                'Amélioration_Cohérence_%': metrics.get('coherence_improvement_%', 0),                'Réduction_Std_Mean': metrics.get('mean_std_reduction', 0)            })        if not comparison_data:        print("⚠ Aucune donnée à analyser")        return        df = pd.DataFrame(comparison_data)        # Sauvegarder le DataFrame détaillé    csv_path = os.path.join(output_dir, 'comparison_metrics.csv')    df.to_csv(csv_path, index=False)    print(f"✓ Métriques détaillées sauvegardées: {csv_path}")        # Mapper les noms de méthodes pour un affichage plus lisible    method_display_names = {        'flat_field': 'Flat-field',        'reinhard': 'Reinhard',        'histogram_matching': 'Histogram Matching',        'macenko': 'Macenko',        'pipeline_ff_reinhard': 'Pipeline (FF+Reinhard)'    }    df['Méthode'] = df['Méthode'].map(lambda x: method_display_names.get(x, x))        # Calculer les moyennes par méthode    summary = df.groupby('Méthode').agg({        'Réduction_Variance_%': 'mean',        'Amélioration_Cohérence_%': 'mean',        'Réduction_Std_Mean': 'mean'    }).round(2)        # Calculer le score combiné    summary['Score_Combiné'] = (        summary['Réduction_Variance_%'] * 0.5 +        summary['Amélioration_Cohérence_%'] * 0.5    ).round(2)        # Trier par score combiné (décroissant)    summary = summary.sort_values('Score_Combiné', ascending=False)        # Sauvegarder le tableau récapitulatif    summary_path = os.path.join(output_dir, 'summary_by_method.csv')    summary.to_csv(summary_path)    print(f"✓ Tableau récapitulatif sauvegardé: {summary_path}")        # Afficher le tableau dans la console    print(f"\n{'='*70}")    print("TABLEAU DES RÉSULTATS GLOBAUX")    print(f"{'='*70}")    print("\nRésumé par méthode (moyennes sur toutes les classes):\n")    print(summary.to_string())    print(f"\n{'='*70}")        # Trouver la meilleure méthode    best_method = summary.index[0]    print(f"\n⭐ MÉTHODE RECOMMANDÉE (meilleur score combiné): {best_method}")    print(f"   - Réduction variance: {summary.loc[best_method, 'Réduction_Variance_%']:.2f}%")    print(f"   - Amélioration cohérence: {summary.loc[best_method, 'Amélioration_Cohérence_%']:.2f}%")    print(f"   - Score combiné: {summary.loc[best_method, 'Score_Combiné']:.2f}")    print(f"{'='*70}\n")        # Générer un rapport texte    report_path = os.path.join(output_dir, 'comparison_report.txt')    with open(report_path, 'w', encoding='utf-8') as f:        f.write("=" * 70 + "\n")        f.write("RAPPORT DE COMPARAISON DES MÉTHODES DE NORMALISATION\n")        f.write("=" * 70 + "\n\n")                f.write("RÉSUMÉ PAR MÉTHODE (Moyennes)\n")        f.write("-" * 70 + "\n")        f.write(summary.to_string())        f.write("\n\n")                f.write("INTERPRÉTATION DES MÉTRIQUES:\n")        f.write("-" * 70 + "\n")        f.write("1. Réduction_Variance_%: Plus élevé = mieux\n")        f.write("   → Indique à quel point la variance entre images est réduite\n\n")        f.write("2. Amélioration_Cohérence_%: Plus élevé = mieux\n")        f.write("   → Indique à quel point les statistiques (mean) sont cohérentes\n\n")        f.write("3. Score_Combiné: Plus élevé = mieux\n")        f.write("   → Score = (Réduction_Variance_% × 0.5) + (Amélioration_Cohérence_% × 0.5)\n\n")                f.write("RECOMMANDATION:\n")        f.write("-" * 70 + "\n")        f.write(f"MÉTHODE RECOMMANDÉE: {best_method}\n")        f.write(f"  - Réduction variance: {summary.loc[best_method, 'Réduction_Variance_%']:.2f}%\n")        f.write(f"  - Amélioration cohérence: {summary.loc[best_method, 'Amélioration_Cohérence_%']:.2f}%\n")        f.write(f"  - Score combiné: {summary.loc[best_method, 'Score_Combiné']:.2f}\n")        print(f"✓ Rapport texte sauvegardé: {report_path}")        # Afficher le tableau dans le notebook    from IPython.display import display    display(summary)print("✓ Fonction de génération du tableau global définie")

## 10. Exécution du test de normalisation

In [ ]:
# Trouver toutes les classes dans train_valclasses = find_all_classes(TRAIN_VAL_PATH)if not classes:    print(f"❌ Aucune classe trouvée dans {TRAIN_VAL_PATH}")else:    print(f"✓ Classes trouvées: {list(classes.keys())}\n")        # Créer le répertoire de sortie    os.makedirs(OUTPUT_DIR, exist_ok=True)        # Dictionnaire pour stocker toutes les statistiques    all_stats = defaultdict(dict)        # Tester chaque classe    for class_name, class_dir in sorted(classes.items()):        class_metrics = test_class_samples(            class_name=class_name,            class_dir=class_dir,            output_dir=OUTPUT_DIR,            max_samples=SAMPLES_PER_CLASS,            flat_field_sigma=50.0,            return_stats=True        )        if class_metrics:            all_stats[class_name] = class_metrics        print(f"\n{'='*70}")    print(f"✅ TEST DES IMAGES TERMINÉ!")    print(f"Résultats sauvegardés dans: {OUTPUT_DIR}")    print(f"{'='*70}\n")        # Générer le tableau des résultats globaux avec score combiné    if all_stats:        generate_global_comparison_table(all_stats, OUTPUT_DIR)    else:        print("⚠ Aucune statistique collectée")

## 11. Visualisation des résultats

In [ ]:
from IPython.display import Image, display# Afficher un exemple de visualisationif os.path.exists(OUTPUT_DIR):    # Trouver la première classe    classes_found = [d for d in os.listdir(OUTPUT_DIR)                      if os.path.isdir(os.path.join(OUTPUT_DIR, d))]        if classes_found:        first_class = classes_found[0]        class_dir = os.path.join(OUTPUT_DIR, first_class)                # Trouver la première image de comparaison        comparison_files = [f for f in os.listdir(class_dir)                            if f.endswith('_comparison.png')]                if comparison_files:            example_path = os.path.join(class_dir, comparison_files[0])            print(f"Exemple de visualisation ({first_class}):")            display(Image(example_path))        else:            print("Aucune visualisation trouvée")    else:        print("Aucune classe trouvée dans les résultats")

## 12. Téléchargement des résultats

In [ ]:
# Créer un ZIP des résultats
import subprocess
subprocess.run(["zip", "-r", "/content/results_normalization.zip", OUTPUT_DIR])

# Télécharger
from google.colab import files
files.download('/content/results_normalization.zip')

print("✓ Résultats téléchargés")